In [3]:
import pandas as pd
import numpy as np
import random
import matplotlib as plt
import os
import math


In [4]:
cities = pd.read_excel(r'C:\Users\Dilsan\Desktop\Coding for Data Science and Data Management\Python\Project\worldcities.xlsx')

In [6]:
def haversine_distance_in_km(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    earth_radius = 6371  # You can use 3959 for miles

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Differences in latitude and longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = earth_radius * c

    return distance



In [7]:
cities = cities[cities['lat']>30].reset_index().drop(['index'],axis=1)

In [8]:
london_lat = 51.5072
london_lng = -0.1275

In [9]:
cities['distance_km'] = cities.apply(lambda row: haversine_distance_in_km(london_lat, london_lng, row['lat'], row['lng']), axis=1)
df = cities.sort_values('distance_km')

In [10]:
def check_population(population):
    if population > 200000:
        return 2
    else:
        return 0

In [11]:
def check_country(current_city_country,destination_country):
    if current_city_country != destination_country:
        return 2
    else:
        return 0
    

In [12]:
def get_travel_cost(index):
    res_dict = {0:2,1:4,2:8}
    return res_dict[index]

In [31]:
def calculate_cost(current_city_country,current_city_id,df):
    #current_city_country = df[df['city_id']==current_city_id]['country'].values[0]
    #df = df[df['city_id']!=current_city_id]
    #df = df.reset_index().drop('index',axis=1)
#DataFrame df called 'cost_country'.    
#applies a function to each row or column of the DataFrame
#lambda function takes each row of the DataFrame (row) and calls the check_country function with current_city_country 
#and the value in the 'country', 'population' columns of that row.
#axis=1: lambda function should be applied to rows. If axis=0 (or not specified), it would apply the function to columns.
    df['cost_country'] = df.apply(lambda row: check_country(current_city_country,row['country']), axis=1)
    #creates a new column named 'cost_country'
    df['cost_pop'] = df.apply(lambda row: check_population(row['population']), axis=1)
    df['travel_cost'] = df.apply(lambda row: get_travel_cost(row.name), axis=1)


    return df
    

In [32]:
def sum_total_cost(row):
    return row['cost_country'] + row['cost_pop'] + row['travel_cost']



In [33]:
def choose_optimum_city(df):
    
    df = df.sort_values(by=['lng'], ascending=[False]).reset_index().loc[0]
    city_id = df['city_id']
    total_cost = df['total_cost']
    city_name = df['city_name']
    lat = df['lat']
    lng = df['lng']
    country= df['country']
    return {'city_id':city_id,'total_cost':total_cost,'city_name':city_name,'lat':lat,'lng':lng,'country':country}


In [34]:

df = df.rename(columns={'id':'city_id','city':'city_name'})
df = df.reset_index().drop('index',axis=1)
df

,city_name,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,city_id,distance_km
0,London,London,51.5072,-0.1275,United Kingdom,GB,GBR,"London, City of",primary,10979000.0,1826645935,0.000000
1,Holborn,Holborn,51.5172,-0.1182,United Kingdom,GB,GBR,Camden,NaN,13023.0,1826657058,1.284766
2,Lambeth,Lambeth,51.4903,-0.1193,United Kingdom,GB,GBR,Lambeth,NaN,9675.0,1826545364,1.963051
3,Spitalfields,Spitalfields,51.5166,-0.0750,United Kingdom,GB,GBR,Tower Hamlets,NaN,10286.0,1826499077,3.780491
4,Kensington,Kensington,51.5000,-0.1900,United Kingdom,GB,GBR,Kensington and Chelsea,NaN,64681.0,1826583042,4.399398
...,...,...,...,...,...,...,...,...,...,...,...,...
19382,Katsuura,Katsuura,35.1525,140.3211,Japan,JP,JPN,Chiba,NaN,17324.0,1392044206,9636.254742
19383,Kisanuki,Kisanuki,31.3447,130.9453,Japan,JP,JPN,Kagoshima,NaN,14495.0,1392204633,9638.538863
19384,Kushima,Kushima,31.4645,131.2284,Japan,JP,JPN,Miyazaki,NaN,17363.0,1392989065,9639.374565
19385,Nishinoomote,Nishinoomote,30.7325,130.9975,Japan,JP,JPN,Kagoshima,NaN,14980.0,1392981409,9700.971905


In [36]:
import random
random.seed(42)  # For reproducibility, we can change the seed or remove it
sampled_df = pd.concat([df.iloc[0:1], df.iloc[1:].sample(n=499, random_state=42)], ignore_index=True)
sampled_df = sampled_df.sort_values(by='distance_km')
sampled_df = sampled_df.reset_index().drop('index',axis=1)
sampled_df

,city_name,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,city_id,distance_km
0,London,London,51.5072,-0.1275,United Kingdom,GB,GBR,"London, City of",primary,10979000.0,1826645935,0.000000
1,Kensington,Kensington,51.5000,-0.1900,United Kingdom,GB,GBR,Kensington and Chelsea,NaN,64681.0,1826583042,4.399398
2,Wanstead,Wanstead,51.5778,0.0286,United Kingdom,GB,GBR,Redbridge,NaN,11543.0,1826002020,13.347853
3,East Barnet,East Barnet,51.6430,-0.1630,United Kingdom,GB,GBR,Barnet,NaN,16137.0,1826125584,15.298259
4,Pinner,Pinner,51.5932,-0.3894,United Kingdom,GB,GBR,Harrow,NaN,31130.0,1826466478,20.478690
...,...,...,...,...,...,...,...,...,...,...,...,...
495,Yuasa,Yuasa,34.0331,135.1786,Japan,JP,JPN,Wakayama,NaN,11340.0,1392420914,9552.428391
496,Kamirenjaku,Kamirenjaku,35.6833,139.5594,Japan,JP,JPN,Tōkyō,NaN,192489.0,1392889022,9554.535603
497,Zama,Zama,35.4886,139.4075,Japan,JP,JPN,Kanagawa,NaN,130608.0,1392313741,9568.783454
498,Makinohara,Makinohara,34.7400,138.2247,Japan,JP,JPN,Shizuoka,NaN,44111.0,1392331994,9600.638815


In [39]:
def calculate_cost(current_city_country,current_city_id,df):
    #current_city_country = df[df['city_id']==current_city_id]['country'].values[0]
    #df = df[df['city_id']!=current_city_id]
    df = df.reset_index().drop('index',axis=1)
    
    df['cost_country'] = df.apply(lambda row: check_country(current_city_country,row['country']), axis=1)
    df['cost_pop'] = df.apply(lambda row: check_population(row['population']), axis=1)
    df['travel_cost'] = df.apply(lambda row: get_travel_cost(row.name), axis=1)


    return df
    

In [40]:

city_ids = sampled_df.city_id.to_list()
info_cities = {}

for each_city_id in city_ids:
    lat = sampled_df[sampled_df['city_id']==each_city_id].lat.values[0]
    lng = sampled_df[sampled_df['city_id']==each_city_id].lng.values[0]    
    name = sampled_df[sampled_df['city_id']==each_city_id].city_name.values[0]
    country_name = sampled_df[sampled_df['city_id']==each_city_id].country.values[0]
    
    sampled_df['distance_km'] = sampled_df.apply(lambda row: haversine_distance_in_km(lat,lng, row['lat'], row['lng']), axis=1)
    df_sorted = sampled_df.sort_values(by='distance_km').reset_index().drop('index',axis=1)
    nearest_three = df_sorted[1:4]
    cost_df = calculate_cost(country_name,each_city_id,nearest_three)
    sum_df_calculated = cost_df.assign(total_cost=cost_df.apply(sum_total_cost, axis=1))
    city_info = {
        'city_names': nearest_three['city_name'].to_list(),
        'city_ids': nearest_three['city_id'].to_list(),
        'city_distances': nearest_three['distance_km'].to_list(),
        'city_latitudes':  nearest_three['lat'].to_list(),
        'city_longitudes':  nearest_three['lng'].to_list(),
        'city_pops': nearest_three['population'].to_list(),
        'city_countries': nearest_three['country'].to_list(),
        'city_total_cost' : sum_df_calculated['total_cost'].to_list()
    }
    
    info_cities[each_city_id] = city_info

In [52]:
info_cities


{1826645935: {'city_names': ['Kensington', 'Wanstead', 'East Barnet'],
  'city_ids': [1826583042, 1826002020, 1826125584],
  'city_distances': [4.399397763981987,
   13.347852752671828,
   15.298259010581251],
  'city_latitudes': [51.5, 51.5778, 51.643],
  'city_longitudes': [-0.19, 0.0286, -0.163],
  'city_pops': [64681.0, 11543.0, 16137.0],
  'city_countries': ['United Kingdom', 'United Kingdom', 'United Kingdom'],
  'city_total_cost': [2, 4, 8]},
 1826583042: {'city_names': ['London', 'East Barnet', 'Pinner'],
  'city_ids': [1826645935, 1826125584, 1826466478],
  'city_distances': [4.399397763981987, 16.009991478236856, 17.24877415883182],
  'city_latitudes': [51.5072, 51.643, 51.5932],
  'city_longitudes': [-0.1275, -0.163, -0.3894],
  'city_pops': [10979000.0, 16137.0, 31130.0],
  'city_countries': ['United Kingdom', 'United Kingdom', 'United Kingdom'],
  'city_total_cost': [4, 4, 8]},
 1826002020: {'city_names': ['London', 'East Barnet', 'Kensington'],
  'city_ids': [1826645935, 

In [64]:
city_ids = sampled_df.city_id.to_list()
info_cities = {}

for each_city_id in city_ids:
    break

In [65]:
each_city_id

1826645935

In [66]:
lat = sampled_df[sampled_df['city_id']==each_city_id].lat.values[0]
lng = sampled_df[sampled_df['city_id']==each_city_id].lng.values[0]    
name = sampled_df[sampled_df['city_id']==each_city_id].city_name.values[0]
country_name = sampled_df[sampled_df['city_id']==each_city_id].country.values[0]

In [67]:
sampled_df['distance_km'] = sampled_df.apply(lambda row: haversine_distance_in_km(lat,lng, row['lat'], row['lng']), axis=1)
df_sorted = sampled_df.sort_values(by='distance_km').reset_index().drop('index',axis=1)
nearest_three = df_sorted[1:4]
cost_df = calculate_cost(country_name,each_city_id,nearest_three)
sum_df_calculated = cost_df.assign(total_cost=cost_df.apply(sum_total_cost, axis=1))
sum_df_calculated

,city_name,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,city_id,distance_km,cost_country,cost_pop,travel_cost,total_cost
0,Kensington,Kensington,51.5000,-0.1900,United Kingdom,GB,GBR,Kensington and Chelsea,NaN,64681.0,1826583042,4.399398,0,0,2,2
1,Wanstead,Wanstead,51.5778,0.0286,United Kingdom,GB,GBR,Redbridge,NaN,11543.0,1826002020,13.347853,0,0,4,4
2,East Barnet,East Barnet,51.6430,-0.1630,United Kingdom,GB,GBR,Barnet,NaN,16137.0,1826125584,15.298259,0,0,8,8


In [68]:
lat = sampled_df[sampled_df['city_id']==each_city_id].lat.values[0]
lng = sampled_df[sampled_df['city_id']==each_city_id].lng.values[0]    
name = sampled_df[sampled_df['city_id']==each_city_id].city_name.values[0]
country_name = sampled_df[sampled_df['city_id']==each_city_id].country.values[0]

sampled_df['distance_km'] = sampled_df.apply(lambda row: haversine_distance_in_km(lat,lng, row['lat'], row['lng']), axis=1)
df_sorted = sampled_df.sort_values(by='distance_km').reset_index().drop('index',axis=1)
nearest_three = df_sorted[1:4]
cost_df = calculate_cost(country_name,each_city_id,nearest_three)
sum_df_calculated = cost_df.assign(total_cost=cost_df.apply(sum_total_cost, axis=1))
city_info = {
    'city_names': nearest_three['city_name'].to_list(),
    'city_ids': nearest_three['city_id'].to_list(),
    'city_distances': nearest_three['distance_km'].to_list(),
    'city_latitudes':  nearest_three['lat'].to_list(),
    'city_longitudes':  nearest_three['lng'].to_list(),
    'city_pops': nearest_three['population'].to_list(),
    'city_countries': nearest_three['country'].to_list(),
    'city_total_cost' : sum_df_calculated['total_cost'].to_list()
}

In [69]:
info_cities

{}

In [70]:
len(info_cities)

0

In [71]:
info_cities[1826645935]

KeyError: 1826645935

In [72]:
total_cost = 0
coordinates_lat = []
coordinates_lng = []
city_and_country = []

for each_city in info_cities.keys():
    eastest_city_index = info_cities[each_city]['city_longitudes'].index(max(info_cities[each_city]['city_longitudes']))
    eastest_city_id = info_cities[each_city]['city_ids'][eastest_city_index]
    eastest_city_cost = info_cities[each_city]['city_total_cost'][eastest_city_index]
    eastest_city_lat = info_cities[each_city]['city_latitudes'][eastest_city_index]
    eastest_city_lng = info_cities[each_city]['city_longitudes'][eastest_city_index]
    eastest_city_country = info_cities[each_city]['city_countries'][eastest_city_index]
    eastest_city_name= info_cities[each_city]['city_names'][eastest_city_index]
    total_cost += eastest_city_cost
    coordinates_lat.append(eastest_city_lat)
    coordinates_lng.append(eastest_city_lng)
    city_and_country.append(eastest_city_country+'_'+eastest_city_name)
    
coordinates = zip(coordinates_lat,coordinates_lng)
    

In [73]:
city_and_country

[]

In [74]:

import folium



# Create a map centered at a specific location
m = folium.Map(location=[0, 0], zoom_start=2)

# Add markers for each coordinate
for coord in coordinates:
    folium.Marker(coord).add_to(m)

# Save the map to an HTML file or display it in a Jupyter Notebook
m.save("map.html")

In [75]:
total_cost/24


0.0